langchain -  integrate things with LLM

In [ ]:
from google.colab import runtime
runtime.unassign()

In [1]:
pip install langchain

langsmith - to trace/monitor application

In [2]:
LANGCHAIN_TRACING_V2 = 'true'
LANGCHAIN_ENDPOINT = "https://api.smith.langchain.com"

create passkey for APIs

In [3]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


Groq

In [4]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 25.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


Groq API

In [5]:
os.environ['GROQ_API_KEY'] = getpass.getpass()

··········


model

In [6]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama3-8b-8192")

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [SystemMessage(content="Translate the following from English to Tamil"),
            HumanMessage(content="Hi!"),
            ]
model.invoke(messages)

AIMessage(content='வணக்கம்! (Vanakkam!)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 24, 'total_tokens': 42, 'completion_time': 0.015, 'prompt_time': 0.000800376, 'queue_time': 0.013863873, 'total_time': 0.015800376}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-efa89dd0-089d-444d-b1fa-931c0b68833b-0', usage_metadata={'input_tokens': 24, 'output_tokens': 18, 'total_tokens': 42})

extract output

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [9]:
result = model.invoke(messages)

In [10]:
parser.invoke(result)

'வணக்கம்! (Vanakkam!)'

chaining with different tools

In [11]:
chain = model | parser

In [12]:
chain.invoke(messages)

'வணக்கம்! (Vanaakkam!)'

prompt template
language: The language to translate into
text: The content

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
system_template = "Translate the following text into {language}:"

In [15]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [16]:
result = prompt_template.invoke({'language': "Hindi", 'text': "How are you?"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following text into Hindi:', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})])

chain

In [17]:
chain = prompt_template | model | parser

In [18]:
chain.invoke({"language": "Hindi", "text": 'How are you?'})

'You are asking me to translate "How are you?" into Hindi.\n\nThe translation of "How are you?" in Hindi is: क्या हाल है? (Kya haal hai?)\n\nNote: क्या (kya) means "what" or "how", हाल (haal) means "condition" or "state", and है (hai) is the verb "to be" in the present tense.'

In [19]:
chain.invoke({"language": "Marathi", "text": 'How are you?'})

'In Marathi, "How are you?" can be translated to:\n\nकसा आहेस? (Kasā āhēs?)\n\nHere\'s a breakdown:\n\n* कसा (Kasā) means "how"\n* आहेस (āhēs) is the second person singular present tense of the verb "to be", which means "are you"\n\nSo, कसा आहेस? is a common greeting in Marathi, used to ask about someone\'s well-being or health.'

In [20]:
chain.invoke({"language": "Gujarati", "text": 'How are you?'})

'The translation of "How are you?" in Gujarati is:\n\nતમે કેમ છો? (Tame kem cho?)'

retrieve data from document
1. Documents
2. Vector Stores
3. Retrievers

In [21]:
pip install langchain-chroma langchain-nomic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

Langchain Document abstract has 2 components:
page_content: a string representing the content
metadata: a dict of metadata

In [30]:


from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={'source': "mammal-pets-doc"},
    ),
    Document(
        page_content="Lion is the kind of the Jungle and it has a group of Pride.",
        metadata={'source': 'predators-doc'},

    ),
    Document(
        page_content="Peacock are intelligent birds and is the national bird of India",
        metadata={'source': 'bird-pets-doc'},
    ),
]


In [23]:
os.environ['NOMIC_API_KEY'] = getpass.getpass()

··········


In [24]:
from langchain_chroma import Chroma
from langchain_nomic import NomicEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=NomicEmbeddings(model="nomic-embed-text-v1.5")
)

In [25]:
vectorstore.similarity_search("Lion")

[Document(metadata={'source': 'predators-doc'}, page_content='Lion is the kind of the Jungle and it has a group of Pride.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Peacock are intelligent birds and is the national bird of India'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [26]:
vectorstore.similarity_search_with_relevance_scores("Lion")

[(Document(metadata={'source': 'predators-doc'}, page_content='Lion is the kind of the Jungle and it has a group of Pride.'),
  0.7156161837948953),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Peacock are intelligent birds and is the national bird of India'),
  0.4433571729694046),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.37175730592897616)]

In [32]:
from langchain_core.runnables import RunnableLambda

'''
def add_one(x):
  return x+1
runnable = RunnableLambda(add_one)
runnable.batch([1,2,3])
'''

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['owl', 'shark'])

[[Document(metadata={'source': 'bird-pets-doc'}, page_content='Peacock are intelligent birds and is the national bird of India')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages(['human', message])
rag_chain = {'context': retriever, 'question': RunnablePassthrough()} | prompt | model

In [34]:
response = rag_chain.invoke("tell me about lions")
print(response.content)

According to the provided context, here's what I can tell you about lions:

* Lions are the kind of the jungle.
* They have a group called Pride.
